# Alpha–Phi Binary Explorer (v8) — *Fibonacci Offset Test*

**Key change:** after running the adaptive iterate **and** the hybrid ping‑pong,  
we **subtract** the original Fibonacci fraction `seed` (interpreted as `0.[bit‑string]`) from the final value:

```python
x_adjusted = x_final - seed
```

The idea: remove the “raw binary scaffold” and see whether the *residual* falls closer to α.

Search grid is the same as v7, but we compute relative error on `x_adjusted`.  Early‑exit on first `< 1 %` hit (or `< 0.1 %` if lucky).

In [1]:
import mpmath as mp, csv, itertools, math, sys
mp.mp.dps = 300
ALPHA = mp.mpf('7.2973525693e-3')
PHI   = (1 + mp.sqrt(5)) / 2
def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]
def fib_word_real(n):
    bits=fib_word_bits(n)
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(bits))


In [2]:
def iterate_adapt(x0, steps, p, q, lam, gamma):
    for _ in range(steps):
        eta = gamma*(x0/ALPHA-1)
        core=1/((10**(q+eta))*x0**p)
        x0 = core**lam
    return x0

In [3]:
def hybrid_tuned(x0, cycles, q, comp_p, phi_scale):
    for _ in range(cycles):
        x0 = (phi_scale*PHI)/x0
        x0 = 1/(10**q * x0**comp_p)
    return x0

## Sweep with Fibonacci offset

In [4]:
hits=[]
with open('scan_log_v8.csv','w',newline='') as f:
    w=csv.writer(f); w.writerow(['p','q','lam','gamma','phi_scale','bits','rel_err'])
    space=itertools.product(
        (0.5,0.7,1.0,1.3,1.6),
        (0.7,1.0,1.3),
        (0.01,0.03,0.06),
        (0.03,0.05,0.07),
        (0.75,0.85,0.95,1.05),
        (512,1024)
    )
    for p,q,lam,gamma,phi_scale,bits in space:
        seed=fib_word_real(bits)
        x = iterate_adapt(seed,60,p,q,lam,gamma)
        x = hybrid_tuned(x,15,q,p,phi_scale)
        x_adj = x - seed
        rel = abs((x_adj-ALPHA)/ALPHA)
        if rel < 0.01:
            hits.append(rel)
            w.writerow([p,q,lam,gamma,phi_scale,bits, str(rel)])
            print('Hit <1%:', rel, 'params', (p,q,lam,gamma,phi_scale,bits))
            if rel<0.001 or len(hits)>=3:
                break
    else:
        print('No hit <1% in space')


No hit <1% in space
